<a href="https://colab.research.google.com/github/hhmida/ESB/blob/main/Spark_ML_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pipeline avec Spark ML
---

Le pipeline permet d'organiser les différentes transformations et prétraitements sur les données jusqu'à la création d'un modèle. Il facilite ainsi sa configuration et sa réutilisation. Un pipeline est composé de stages de **transformers** et **estimators**. 

**Transformers**

Il transforme un DataFrame en un nouveau DataFrame en lui ajoutant généralement une ou plusieurs colonnes. Les transformers commence par *fit()* sur le DataFrame et possèdent aussi une méthode *transform()* pour transformer le DataFrame.

Exemple : StandardScaler
```python
from pyspark.ml.feature import StandardScaler
dataFrame = spark.read.csv("data.csv")
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",
                        withStd=True, withMean=False)
scalerModel = scaler.fit(dataFrame)
scaledData = scalerModel.transform(dataFrame)
scaledData.show()
```

**Estimators**

C'est une abstraction d'un algorithme d'apprentissage qui sera entrainé sur un dataset. Sa méthode *fit()* dont le paramètre est un DataFrame produit un modèle. Ce modèle peut être utilisé ensuite pour transformer (méthode *transform()*) des données afin d'appliquer le modèle pour prédire leur classe par exemple.

Exemple : LinearRegression

```python
from pyspark.ml.regression import LinearRegression

#charger et préparer le dataset
# ...
# Créer le modèle
lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
lrModel = lr.fit(trainingDF)

# Appliquer  sur les données de test
lrModel.transform(testDF)
```
Nous allons appliquer un pipeline pour un problème de classification afin de prédire si un vol serait en retard ou non.

### Installer Pyspark télécharger le dataset



In [1]:
!pip install pyspark  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 37 kB/s 
     |████████████████████████████████| 199 kB 62.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=c6e2ff38214367682111fd7fe26dbb6125525724aee4d6dabdf8922b9da46abb
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
!wget https://github.com/hhmida/datasets/raw/master/flights.csv

--2022-12-12 14:14:58--  https://github.com/hhmida/datasets/raw/master/flights.csv
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hhmida/datasets/master/flights.csv [following]
--2022-12-12 14:14:58--  https://raw.githubusercontent.com/hhmida/datasets/master/flights.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17698856 (17M) [text/plain]
Saving to: ‘flights.csv’

flights.csv         100%[===================>]  16.88M  --.-KB/s    in 0.1s    

2022-12-12 14:14:59 (124 MB/s) - ‘flights.csv’ saved [17698856/17698856]



## Importer les librairies Spark SQL et Spark ML 

In [3]:
from pyspark.sql import SparkSession
spark =  SparkSession.builder.appName('Spark ML').getOrCreate()

# Pour créer un pipeline
from pyspark.ml import Pipeline

# Estimator de type Régression logistique pour la classification
from pyspark.ml.classification import LogisticRegression

# La préparation des données pour la regression logistique
from pyspark.ml.feature import VectorAssembler, StringIndexer, MinMaxScaler

## Charger et explorer le dataset

Notre dataset est sous forme de fichier csv **flights.csv**

In [4]:
flightSchema = "DayofMonth int,DayOfWeek int,Carrier string,OriginAirportID string,DestAirportID string, DepDelay int, ArrDelay int, Late int"
data = spark.read.csv('flights.csv', schema=flightSchema, header=True)
data.show()

+----------+---------+-------+---------------+-------------+--------+--------+----+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|Late|
+----------+---------+-------+---------------+-------------+--------+--------+----+
|        21|        2|     WN|          10721|        13342|      26|      57|   1|
|        13|        1|     AA|          15016|        12892|      51|      27|   1|
|         5|        5|     FL|          10397|        11433|       9|       4|   0|
|        22|        1|     US|          11278|        14100|      35|      71|   1|
|        23|        4|     WN|          12451|        10693|       9|       5|   0|
|         5|        7|     AA|          11298|        15016|      39|      42|   1|
|         4|        6|     UA|          13930|        14307|      71|      58|   1|
|        10|        3|     9E|          14307|        11433|      68|     140|   1|
|        29|        7|     UA|          14057|        14771|     130|     12

In [5]:
data.count()

637083

In [6]:
from pyspark.sql.functions import *
data.groupby().agg(min("DepDelay"),max("DepDelay"),mean("DepDelay"),min("ArrDelay"),max("ArrDelay"),mean("ArrDelay")).show()

+-------------+-------------+-----------------+-------------+-------------+-----------------+
|min(DepDelay)|max(DepDelay)|    avg(DepDelay)|min(ArrDelay)|max(ArrDelay)|    avg(ArrDelay)|
+-------------+-------------+-----------------+-------------+-------------+-----------------+
|          -52|          149|28.89959706976956|          -86|          149|29.31866177562421|
+-------------+-------------+-----------------+-------------+-------------+-----------------+



In [7]:
from pyspark.ml.stat import Correlation
va = VectorAssembler(inputCols=["DayofMonth", "DayOfWeek", "DepDelay", "ArrDelay","Late"], outputCol="features")

In [9]:
newData =va.transform(data)

In [10]:
newData.show()

+----------+---------+-------+---------------+-------------+--------+--------+----+--------------------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|Late|            features|
+----------+---------+-------+---------------+-------------+--------+--------+----+--------------------+
|        21|        2|     WN|          10721|        13342|      26|      57|   1|[21.0,2.0,26.0,57...|
|        13|        1|     AA|          15016|        12892|      51|      27|   1|[13.0,1.0,51.0,27...|
|         5|        5|     FL|          10397|        11433|       9|       4|   0|[5.0,5.0,9.0,4.0,...|
|        22|        1|     US|          11278|        14100|      35|      71|   1|[22.0,1.0,35.0,71...|
|        23|        4|     WN|          12451|        10693|       9|       5|   0|[23.0,4.0,9.0,5.0...|
|         5|        7|     AA|          11298|        15016|      39|      42|   1|[5.0,7.0,39.0,42....|
|         4|        6|     UA|          13930|        1

In [11]:
r1 = Correlation.corr(newData, "features").head()
print("Pearson correlation matrix:\n" + str(r1[0]))

Pearson correlation matrix:
DenseMatrix([[ 1.        ,  0.02932506,  0.00345172,  0.00521996,  0.00760964],
             [ 0.02932506,  1.        , -0.00797621, -0.01716528, -0.01259032],
             [ 0.00345172, -0.00797621,  1.        ,  0.89696607,  0.70494444],
             [ 0.00521996, -0.01716528,  0.89696607,  1.        ,  0.80351383],
             [ 0.00760964, -0.01259032,  0.70494444,  0.80351383,  1.        ]])


## Découper le dataset en training set et test set
Dans ce qui suit 70% des données pour le training et 30% pour le test.

In [14]:
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print ("Training Rows:", train_rows, " Testing Rows:", test_rows)

Training Rows: 446788  Testing Rows: 190295


## Définir le Pipeline
Les étapes d'un pipeline dépendent de type d'apprentissage (supervisé, non supervisé, ...) et le type du problème et l'algorithme utilisé (classification, régression linéaire, arbre de décision, ...).

Dans cet exemple, il s'agit de problème de classification avec l'algorithme de régression logistique. Alors, le pipeline est constitué des éléments suivants :

- **StringIndexer** pour transformer les colonnes catégoriques en colonne numériques.
- **VectorAssembler** pour créer un vecteur des valeurs numériques continues.
- **MinMaxScaler** pour normaliser les valeurs numériques dans l'intervalle [0,1].
- **VectorAssembler** pour regrouper les colonnes catégoriques et numériques en une seule colonne vecteur.
- **LogisticRegression** l'algorithme d'apprentissage pour le modèle de classification.

In [12]:
monthdayIndexer = StringIndexer(inputCol="DayofMonth", outputCol="DayofMonthIdx")
weekdayIndexer = StringIndexer(inputCol="DayOfWeek", outputCol="DayOfWeekIdx")
carrierIndexer = StringIndexer(inputCol="Carrier", outputCol="CarrierIdx")
originIndexer = StringIndexer(inputCol="OriginAirportID", outputCol="OriginAirportIdx")
destIndexer = StringIndexer(inputCol="DestAirportID", outputCol="DestAirportIdx")
numVect = VectorAssembler(inputCols = ["DepDelay"], outputCol="numFeatures")
minMax = MinMaxScaler(inputCol = numVect.getOutputCol(), outputCol="normNums")
featVect = VectorAssembler(inputCols=["DayofMonthIdx", "DayOfWeekIdx", "CarrierIdx", "OriginAirportIdx", "DestAirportIdx", "normNums"], outputCol="features")
lr = LogisticRegression(labelCol="Late", featuresCol="features")
pipeline = Pipeline(stages=[monthdayIndexer, weekdayIndexer, carrierIndexer, originIndexer, destIndexer, numVect, minMax, featVect, lr])

### Exécuter le Pipeline sur le training set


In [15]:
piplineModel = pipeline.fit(train)
print ("Pipeline complete!")

Pipeline complete!


## Tester le modèle sur le test set

In [16]:
prediction = piplineModel.transform(test)
predicted = prediction.select("features", col("prediction").cast("Int"), col("Late").alias("trueLabel"))
predicted.show(100, truncate=False)

+---------------------------------------------+----------+---------+
|features                                     |prediction|trueLabel|
+---------------------------------------------+----------+---------+
|[25.0,2.0,10.0,11.0,12.0,0.4427860696517413] |1         |1        |
|[25.0,2.0,10.0,18.0,12.0,0.2288557213930348] |0         |1        |
|[25.0,2.0,10.0,37.0,54.0,0.3383084577114428] |0         |1        |
|[25.0,2.0,10.0,37.0,19.0,0.5522388059701493] |1         |1        |
|[25.0,2.0,10.0,37.0,9.0,0.22388059701492538] |0         |0        |
|[25.0,2.0,10.0,37.0,22.0,0.24378109452736318]|0         |1        |
|[25.0,2.0,10.0,37.0,22.0,0.9950248756218906] |1         |1        |
|[25.0,2.0,10.0,37.0,7.0,0.3582089552238806]  |1         |0        |
|[25.0,2.0,10.0,37.0,30.0,0.27860696517412936]|0         |0        |
|[25.0,2.0,10.0,37.0,49.0,0.2537313432835821] |0         |0        |
|[25.0,2.0,10.0,37.0,1.0,0.5572139303482587]  |1         |1        |
|[25.0,2.0,10.0,37.0,1.0,0.6368159

## Évaluer le modèle

### Matrice de confusion
Elle calcule le nombre de :
- True Positives (TP)
- True Negatives (TN)
- False Positives (FP)
- False Negatives (FN)

Plusieurs mesures sont calculées à partir de cette matrice comme :
$$precision = \frac{TP}{TP+FP}$$
$$recall(TPR) = \frac{TP}{TP+FN}$$
$$Accuracy = \frac{TP+TN}{TP+TN+FP+FN}$$

In [17]:
tp = float(predicted.filter("prediction == 1.0 AND truelabel == 1").count())
fp = float(predicted.filter("prediction == 1.0 AND truelabel == 0").count())
tn = float(predicted.filter("prediction == 0.0 AND truelabel == 0").count())
fn = float(predicted.filter("prediction == 0.0 AND truelabel == 1").count())
metrics = spark.createDataFrame([
 ("TP", tp),
 ("FP", fp),
 ("TN", tn),
 ("FN", fn),
 ("Precision", tp / (tp + fp)),
 ("Recall", tp / (tp + fn))],["metric", "value"])
metrics.show()

+---------+------------------+
|   metric|             value|
+---------+------------------+
|       TP|           88264.0|
|       FP|            7056.0|
|       TN|           81755.0|
|       FN|           13220.0|
|Precision|0.9259756609315988|
|   Recall|0.8697331599069804|
+---------+------------------+



### AUC

In [18]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(labelCol="Late", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
auc = evaluator.evaluate(prediction)
print ("AUC = ", auc)

AUC =  0.9483960616472805
